In [2]:
import gc
import re
import math
import json
import os
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
pd.__version__

'0.23.4'

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split

In [11]:
from mlxtend.regressor import StackingRegressor

In [12]:
import datetime
from datetime import date
from datetime import timedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar

In [13]:
# pip install isoweek
# pip install tsfresh

In [14]:
from isoweek import Week

In [19]:
import pickle

In [20]:
from time import time

In [25]:
from sklearn.externals import joblib 

In [27]:
import warnings
warnings.filterwarnings('ignore')

import sys
np.set_printoptions(threshold=sys.maxsize)

# Preparations:

In [108]:
df = pd.read_csv('CPN_Fractal_fct_468053.txt', sep='|')
df['date'] = df['Period Description Short'].apply(lambda x: pd.to_datetime(x[6:14]) - timedelta(days=2))

df = df[df['Market Display Name'].apply(lambda x: 'CVS' not in x)]
df = df[['Market Display Name', 'date','UPC', 'Units']]
df['Market_UPC'] = df['Market Display Name'] + df['UPC'].astype(str)
df.sort_values(by = ['Market_UPC', 'date'], inplace = True)

### add all rows with dates(if date doesn`t exist, add date with zero 'Unit')

In [95]:
max_date = df['date'].max()

def reindex_from_min_day(df):
    dates = pd.date_range(df.index.min(), max_date, freq='W-THU')
    idx = pd.Index(dates, name='date')
    return df.reindex(idx,fill_value=0)

df_new = df
df_new.set_index('date', inplace = True)
df_new = df_new.groupby('Market_UPC').apply(reindex_from_min_day)
df_new.shape

(36850, 4)

### delete all zero 'Unit' and save only the last one zero row:

In [96]:
df_new.drop('Market_UPC', axis = 1, inplace = True)
df_new.reset_index(inplace = True)

In [97]:
df_new = df_new[(df_new.Units != 0) | (df_new.date == '2018-12-27')]

In [98]:
df_new.head(2)

,Market_UPC,date,Market Display Name,UPC,Units
0,Target City TA201219708420,2018-06-07,Target City TA,201219708420,1.0
2,Target City TA201219708420,2018-06-21,Target City TA,201219708420,1.0


In [ ]:
df_market_upc = df_new[['Market Display Name', 'UPC']]

### add Market_UPC

In [82]:
df_market_upc['Market_UPC'] = df_market_upc['Market Display Name'] + df_market_upc['UPC'].astype(str)

### add max date

In [83]:
max_dates = pd.DataFrame(df[df.Units >0] \
                         .reset_index() \
                         .groupby('Market_UPC').max()['date']) \
                         .reset_index() \
                         .rename(columns = {'date': 'max_date'})
# max_dates_cnt = max_dates.groupby(['date']).count()

In [84]:
df_market_upc = df_market_upc.merge(max_dates, how = 'left', on = 'Market_UPC')

### dead_flag (season_type - no data after 2018)

In [85]:
df_market_upc['dead_reason'] = np.where(df_market_upc.max_date < '2018-01-01', 'season', 'alive')
df_market_upc['dead_season'] = np.where(df_market_upc.max_date < '2018-01-01', 1, 0)

## Add another features:

In [165]:
df_market_upc_pre = df_new[['Market_UPC', 'date']] \
                      .groupby(['Market_UPC'])['date'].apply(list) \
                      .apply(np.diff) 

### dead_flag few observations (date less than 4 )

In [160]:
diff_date_len = df_market_upc_pre.apply(lambda x: len(x))

In [161]:
diff_date_len = pd.DataFrame(diff_date_len).reset_index().rename(columns = {'date': 'date_qty'})

In [251]:
df_market_upc = df_market_upc.merge(diff_date_len, how='left', on = 'Market_UPC')

### dead_flag (data is empty more than 1 season period)

In [252]:
df_market_upc

,Market Display Name,UPC,date_diff,Market_UPC,max_date,dead_reason,dead_season,date_qty
0,Target City TA,3500046280,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target City TA3500046280,2017-03-23,season,0,10
1,Target City TA,201219708420,0 0 14 days 1 7 days 2 7 days 3 ...,Target City TA201219708420,2018-12-27,alive,0,28
2,Target Express TA,201219708420,0 0 14 days 1 28 days 2 14 days 3 7 d...,Target Express TA201219708420,2018-12-20,alive,0,9
3,Target GM Only TA,3500046280,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target GM Only TA3500046280,2017-04-06,season,0,14
4,Target GM Only TA,201219708420,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target GM Only TA201219708420,2018-12-27,alive,0,29
5,Target GM TA,3500046280,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target GM TA3500046280,2017-04-06,season,0,14
6,Target GM TA,201219708420,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target GM TA201219708420,2018-12-27,alive,0,29
7,Target Pfresh TA,2848433070,0 0 126 days,Target Pfresh TA2848433070,2018-09-13,alive,1,2
8,Target Pfresh TA,2848433540,0 0 252 days 1 217 days,Target Pfresh TA2848433540,2018-07-26,alive,1,3
9,Target Pfresh TA,3500046280,0 0 7 days 1 7 days 2 7 day...,Target Pfresh TA3500046280,2018-12-06,alive,0,20


In [244]:
df_market_upc_pre2['days'] = df_market_upc_pre2['days'].apply(lambda x: [0] if len(x)==0 else x)

In [255]:
df_market_upc_pre2['max_gap'] =  df_market_upc_pre2['days'].apply(lambda x: max(x))

In [256]:
df_market_upc_pre2['max_gap_flag'] =  df_market_upc_pre2['days'].apply(lambda x: 1 if (max(x) - x[-1])==0 else 0)

In [257]:
df_market_upc = df_market_upc.merge(df_market_upc_pre2, how='left', on = 'Market_UPC')

In [259]:
df_market_upc.drop(['date', 'days', 'big_gap_dead'], axis = 1, inplace = True)

In [266]:
df_market_upc.columns

Index(['Market Display Name', 'UPC', 'date_diff', 'Market_UPC', 'max_date',
       'dead_reason', 'dead_season', 'date_qty', 'max_gap', 'max_gap_flag'],
      dtype='object')

In [267]:
df_market_upc.drop(['dead_reason', 'dead_season'], inplace = True, axis = 1)

In [268]:
df_market_upc.columns

Index(['Market Display Name', 'UPC', 'date_diff', 'Market_UPC', 'max_date',
       'date_qty', 'max_gap', 'max_gap_flag'],
      dtype='object')

In [269]:
df_market_upc['date_qty_flag'] = np.where(df_market_upc['date_qty']<4,1,0)

In [270]:
df_market_upc.columns

Index(['Market Display Name', 'UPC', 'date_diff', 'Market_UPC', 'max_date',
       'date_qty', 'max_gap', 'max_gap_flag', 'date_qty_flag'],
      dtype='object')

In [271]:
df_market_upc.head()

,Market Display Name,UPC,date_diff,Market_UPC,max_date,date_qty,max_gap,max_gap_flag,date_qty_flag
0,Target City TA,3500046280,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target City TA3500046280,2017-03-23,10,644,1,0
1,Target City TA,201219708420,0 0 14 days 1 7 days 2 7 days 3 ...,Target City TA201219708420,2018-12-27,28,14,0,0
2,Target Express TA,201219708420,0 0 14 days 1 28 days 2 14 days 3 7 d...,Target Express TA201219708420,2018-12-20,9,35,0,0
3,Target GM Only TA,3500046280,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target GM Only TA3500046280,2017-04-06,14,630,1,0
4,Target GM Only TA,201219708420,0 0 7 days 1 7 days 2 7 days 3 7 d...,Target GM Only TA201219708420,2018-12-27,29,7,1,0


In [272]:
df_market_upc.to_csv('add_features.csv')

In [263]:
os.getcwd()

'C:\\Users\\user\\19_10_21_Nielsen'